In [ ]:
%matplotlib inline
%pylab inline

from functools import partial

import menpo.io as mio
from menpo.feature import imgfeature, fast_dsift

from menpo.landmark import labeller, ibug_face_68_trimesh
from menpo.visualize import visualize_images
from menpofit.aam import HolisticAAM

In [ ]:
@imgfeature
def fast_dsift2(image):
    if image.n_channels == 3:
        image = image.as_greyscale(mode='average')
    return fast_dsift(image)

In [ ]:
image_folder = '/vol/atlas/databases/'
model_folder = '/data/papers/alabort_arxiv2015_aam/experiments/'

## Load training data

In [ ]:
training_images = []
for i in mio.import_images(image_folder + 'lfpw/trainset/', 
                           verbose=True, max_images=None):
    i = i.rescale_landmarks_to_diagonal_range(200)
    i = i.crop_to_landmarks_proportion(0.5)
    labeller(i, 'PTS', ibug_face_68_trimesh)
    if i.n_channels == 3:
        training_images.append(i)
        
for i in mio.import_images(image_folder + 'helen/trainset/', 
                           verbose=True, max_images=None):
    i = i.rescale_landmarks_to_diagonal_range(200)
    i = i.crop_to_landmarks_proportion(0.5)
    labeller(i, 'PTS', ibug_face_68_trimesh)
    if i.n_channels == 3:
        training_images.append(i)

In [ ]:
visualize_images(training_images)

## Build and save Active Appearance Model

In [ ]:
features = fast_dsift2
diagonal = 150
scales = (0.5, 1)
max_shape_components= 0.9
max_appearance_components = 0.9

#### AAM

In [ ]:
aam = HolisticAAM(
    training_images, 
    group='ibug_face_68_trimesh', 
    verbose=True,
    features=features,
    diagonal=diagonal,
    scales=scales,
    max_shape_components=max_shape_components,
    max_appearance_components=max_appearance_components)

In [ ]:
mio.export_pickle(aam, model_folder + 'aam.pkl.gz', overwrite=True)